<a href="https://colab.research.google.com/github/sudhirshahu51/RAG/blob/main/Building_Agentic_RAG_with_llamaIndex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing required libraries**

In [ ]:
%%writefile requirements.txt

llama-index==0.10.27
llama-index-llms-text-generation-inference
llama-index-llms-huggingface
llama-index-llms-huggingface-api
#llama-index-llms-openai==0.1.15
#llama-index-embeddings-openai==0.1.7

Writing requirements.txt


In [ ]:
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.6/915.6 kB 40.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently 

**Login for HuggingFace API for LLM**

In [ ]:
import os
from google.colab import userdata
key = userdata.get('HUGGING_FACE')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = key

In [ ]:
from typing import List, Optional

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# This will use the same model, but run remotely on Hugging Face's servers,
# accessed via the Hugging Face Inference API
# Note that using your token will not charge you money,
# the Inference API is free it just has rate limits

remotely_run = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha", token=key)

# Or you can skip providing a token, using Hugging Face Inference API anonymously
remotely_run_anon = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha")

In [ ]:
llm = HuggingFaceInferenceAPI(model_name="HuggingFaceH4/zephyr-7b-alpha", token=key)


In [ ]:
completion_response = llm.complete("To infinity, and")
print(completion_response)


 beyond!

The Toy Story franchise has been a beloved part of pop culture for over two decades, and it's not slowing down anytime soon. The latest installment, Toy Story 4, is set to hit theaters this summer, and it's already generating buzz.

The movie follows the adventures of Woody, Buzz, and the gang as they embark on a new adventure with a new toy, Forky. The trailer for the movie has been released, and it's already getting fans excited for the film.

One of the most exciting things about Toy Story 4 is the return of some beloved characters. Bo Peep, who was last seen in Toy Story 2, is back and looking better than ever. She's now a modern, independent woman, and her new look has been getting a lot of attention.

Another exciting addition to the movie is the introduction of new characters, including Forky, who is voiced by Tony Hale. Forky is a spork with a popsicle stick for a handle, and he's not exactly thrilled about being a toy.

The trailer for Toy Story 4 has been viewed ove

In [ ]:
#https://docs.llamaindex.ai/en/stable/examples/agent/return_direct_agent/
#check examples of using tools using llama index and then apply what have learned inthe tutorial.s

In [10]:
"""For calling own deployed model for inference over huggingface."""

# import os
# from typing import List, Optional

# from llama_index.llms.text_generation_inference import (
#     TextGenerationInference,
# )

# URL = 'https://huggingface.co/HuggingFaceH4/zephyr-7b-alpha'
# model = TextGenerationInference(
#     model_url=URL, token=False
# )  # set token to False in case of public endpoint

# completion_response = model.complete("To infinity, and")
# print(completion_response)

'For calling own deployed model for inference over huggingface.'

In [ ]:
#use llamaindex code for using LLM from hugging face or use hugging face itself